# Демонстрация TF-IDF

Импортируем необходимые библиотеки

In [1]:
import pandas as pd
from IPython.display import display, Math

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import numpy as np
import scipy.sparse as sp
import nltk
from nltk.text import TextCollection

Инициализируем массив ```texts```, который содержит в себе 3 элемента - 3 'синтетических' текста

In [2]:
texts = [u'мама мама мама мыла рама',
         u'рама это рама все просто',
         u'очень просто']

# Считаем частотность

Без использования дополнительных библиотек посчитаем частотность каждого слова по всем текстам

In [3]:
word_to_number = {}
for text in texts:
    for word in text.split():
        word_to_number[word] = word_to_number.get(word, 0) + 1

Для красивой визуализации используем pandas, не обязательно, но наглядно

In [4]:
pd.DataFrame(list(word_to_number.items()))

,0,1
0,мама,3
1,мыла,1
2,рама,3
3,это,1
4,все,1
5,просто,2
6,очень,1


Без использования дополнительных библиотек посчитаем частотность каждого слова по каждому тексту

In [5]:
word_to_number_for_each_texts = {}
for text in texts:
    word_to_number_for_each_texts[text] = {}
    for word in text.split():
        if word_to_number_for_each_texts[text].get(word):
            word_to_number_for_each_texts[text][word] += 1
        else:
            word_to_number_for_each_texts[text][word] = 1

In [6]:
word_to_number_for_each_texts

{'мама мама мама мыла рама': {'мама': 3, 'мыла': 1, 'рама': 1},
 'рама это рама все просто': {'рама': 2, 'это': 1, 'все': 1, 'просто': 1},
 'очень просто': {'очень': 1, 'просто': 1}}

Для красивой визуализации использум pandas, не обязательно, но наглядно (в этот раз для красивой визуализации нужно заморочиться, но мы не будем заморачиваться)

In [7]:
pd.DataFrame(list(word_to_number_for_each_texts.items()))

,0,1
0,мама мама мама мыла рама,"{'мама': 3, 'мыла': 1, 'рама': 1}"
1,рама это рама все просто,"{'рама': 2, 'это': 1, 'все': 1, 'просто': 1}"
2,очень просто,"{'очень': 1, 'просто': 1}"


In [8]:
pd.DataFrame(list(word_to_number_for_each_texts.values()))

,все,мама,мыла,очень,просто,рама,это
0,NaN,3.0,1.0,NaN,NaN,1.0,NaN
1,1.0,NaN,NaN,NaN,1.0,2.0,1.0
2,NaN,NaN,NaN,1.0,1.0,NaN,NaN


Если слова нет в тексте, получаем NaN. Заменим такие значения нулями.

In [9]:
pd.DataFrame(list(word_to_number_for_each_texts.values())).fillna(0)

,все,мама,мыла,очень,просто,рама,это
0,0.0,3.0,1.0,0.0,0.0,1.0,0.0
1,1.0,0.0,0.0,0.0,1.0,2.0,1.0
2,0.0,0.0,0.0,1.0,1.0,0.0,0.0


А теперь воспользуемся библиотекой ```sklearn```

In [10]:
count_vect = CountVectorizer()
temp_matrix = count_vect.fit_transform(texts) # temp_matrix эта промежуточная матрица, понадобится в следующем кейсе,
                                              # для вычисления tf-idf
matrix_counts = temp_matrix.toarray()         # в данной матрице хранятся 
matrix_counts

array([[0, 3, 1, 0, 0, 1, 0],
       [1, 0, 0, 0, 1, 2, 1],
       [0, 0, 0, 1, 1, 0, 0]], dtype=int64)

In [11]:
count_vect.vocabulary_

{'мама': 1, 'мыла': 2, 'рама': 5, 'это': 6, 'все': 0, 'просто': 4, 'очень': 3}

In [12]:
pd.DataFrame(temp_matrix.toarray())

,0,1,2,3,4,5,6
0,0,3,1,0,0,1,0
1,1,0,0,0,1,2,1
2,0,0,0,1,1,0,0


In [ ]:
sorted(list(count_vect.vocabulary_.items()), key=lambda x: x[1])

Визуализируем красиво ```matrix_counts```

In [13]:
words = [x[0] for x in sorted(count_vect.vocabulary_.items(), key=lambda x: x[1])] # список слов, 
                                                                                   # чтобы сделать красивую шапку
pd.DataFrame(matrix_counts, columns=words)  # при создании DataFrame передадим подготовленный список слов

,все,мама,мыла,очень,просто,рама,это
0,0,3,1,0,0,1,0
1,1,0,0,0,1,2,1
2,0,0,0,1,1,0,0


# Что такое TF-IDF?

## TF - term frequency

1) Просто частотность, что мы считали выше (https://en.wikipedia.org/wiki/Tf–idf)

In [14]:
display(Math(r'f_{t,d}'))

<IPython.core.display.Math object>

2) TF по версии российской википедии (https://ru.wikipedia.org/wiki/TF-IDF) (!!!)

In [15]:
display(Math(r'\mathrm{tf}(t,d) = \frac{f_{t,d}}{\sum_k f_{t_k,d}}'))

<IPython.core.display.Math object>

3) Бинарная встречаемость. *0 - не было слова в тексте, 1 - было слово в тексте*

In [16]:
display(Math(r'\mathrm{tf}(t,d) = \left\{ \begin{matrix} 0, f_{t,d} = 0 \\ 1, f_{t,d} ≥ 1 \end{matrix} \right.'))

<IPython.core.display.Math object>

4) Логарифм от частоты +1

In [17]:
display(Math(r'\mathrm{tf}(t,d) = \log(1 + f_{t,d})'))

<IPython.core.display.Math object>

5) Нормированная частота (double normalization)

In [ ]:
display(Math(r'K + (1 - K) \frac { f_{t,d} }{\max({f_{t_i,d}})}'))

Подробности https://en.wikipedia.org/wiki/Tf–idf

## IDF - inverse document frequency

1) 'Стандарт', логарифм отношения числа всех документов к числу документов, содержащих данное слово

In [18]:
display(Math(r'\mathrm{idf}(t,D)=\log\frac{N}{|\{d\in D:t\in d\}|}\
             =\log\frac{|D|}{|(d_{i}\supset t_{i})|}=\log\frac{N}{n_t}'))

<IPython.core.display.Math object>

2) Сглаженный IDF

In [19]:
display(Math(r'\mathrm{idf}(t,D)=\log(1+\frac{N}{n_t})'))

<IPython.core.display.Math object>

3) IDF с использованием максимального значения DF

In [ ]:
display(Math(r'\log\left(1 + \frac {\max_t n_t} {n_t}\right)'))

4) Вероятностная обратная встречаемость

In [ ]:
display(Math(r'\log\frac {N - n_t} {n_t}'))

# Вычислим TF-IDF

## sklearn

Документация http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html#sklearn.feature_extraction.text.TfidfTransformer

Исходный код https://github.com/scikit-learn/scikit-learn/blob/bb592f3865f02f1d6bf9dedce1a2554fa0ada800/sklearn/feature_extraction/text.py#L901

smooth_idf == True: idf = (log((N+1)/(df + 1)) + 1)<br>
smooth_idf == False: idf = (log(N/df) + 1)

In [20]:
tfidf_transformer = TfidfTransformer()
matrix_tfidf = tfidf_transformer.fit_transform(temp_matrix).toarray()
df_index = pd.DataFrame(matrix_tfidf, columns=words)
df_index

,все,мама,мыла,очень,просто,рама,это
0,0.000000,0.922383,0.307461,0.000000,0.000000,0.233832,0.000000
1,0.452123,0.000000,0.000000,0.000000,0.343851,0.687703,0.452123
2,0.000000,0.000000,0.000000,0.795961,0.605349,0.000000,0.000000


In [21]:
pd.DataFrame([tfidf_transformer.idf_]*3, columns=words)  # построим только ```idf```

,все,мама,мыла,очень,просто,рама,это
0,1.693147,1.693147,1.693147,1.693147,1.287682,1.287682,1.693147
1,1.693147,1.693147,1.693147,1.693147,1.287682,1.287682,1.693147
2,1.693147,1.693147,1.693147,1.693147,1.287682,1.287682,1.693147


# Поисковое ранжирование
Ну посчитали мы tf-idf, что с ним делать теперь? Например, это можно использовать как значение релевантности в поиске

In [23]:
import nltk
query = 'просто мама'
q_words = nltk.word_tokenize(query)

In [24]:
q_words

['просто', 'мама']

In [ ]:
[df_index.loc[0][wrd] for wrd in q_words]

In [25]:
arr =[np.asarray([df_index.loc[i][wrd] for wrd in q_words]).sum() for i in range(len(df_index))]
pd.DataFrame(data = np.asarray([texts, arr]).T, columns = ['text', 'rel']).sort_values(by='rel', ascending=False)

,text,rel
0,мама мама мама мыла рама,0.922382964646063
2,очень просто,0.6053485081062916
1,рама это рама все просто,0.34385142961748544


А если база посерьезнее? Поищем что-нибудь про Родю Раскольникова :)
Каждый абзац у нас будет считаться отдельным документом

In [34]:
with open('/tmp/text-mining/dostoevsky.txt', encoding='cp1251') as f:
    book = f.readlines()

In [27]:
%store -r book

In [29]:
count_vect = CountVectorizer()
temp_matrix = count_vect.fit_transform(book) # temp_matrix эта промежуточная матрица, понадобится в следующем кейсе,
                                              # для вычисления tf-idf
matrix_counts = temp_matrix.toarray()         # в данной матрице хранятся 
matrix_counts

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [32]:
book[0]

'    Федор Михайлович Достоевский\n'

In [ ]:
matrix_counts[0].sum()

In [ ]:
temp_matrix.shape

In [30]:
tfidf_transformer = TfidfTransformer()
matrix_tfidf = tfidf_transformer.fit_transform(temp_matrix.toarray()).toarray()
words = [x[0] for x in sorted(count_vect.vocabulary_.items(), key=lambda x: x[1])] # список слов, 

df_index = pd.DataFrame(matrix_tfidf, columns=words)
df_index.head()

,000,10,100000,11,114,12,122,126,127,128,...,ясностью,ясны,ясные,ясный,ясным,ясных,яств,ящик,ящика,ящике
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
query = 'Родион подумал'
q_words = nltk.word_tokenize(query.lower())
q_words

['родион', 'подумал']

In [33]:
arr =[np.asarray([df_index.loc[i][wrd] for wrd in q_words]).sum() for i in range(len(df_index))]
result = pd.DataFrame(data = np.asarray([book, arr]).T, columns = ['text', 'rel']).sort_values(by='rel', ascending=False)

#result['rel'] = result['rel'].apply(lambda x: float(x))
result['rel'] = result['rel'].astype(float)
result = result[result['rel'] > 0]
result.head()

,text,rel
3474,Раскольников подумал с минуту. \n,0.679032
3963,-- Родион Романыч. \n,0.657471
1066,"""А пусть!"" -- подумал он. \n",0.632698
2558,"""Ну, так и есть!"" -- подумал он. \n",0.617722
3265,И он подумал о Соне. \n,0.610471


In [ ]:
len(result)

In [ ]:
result.iloc[0, 1]

## NLTK

Исходный код https://github.com/nltk/nltk/blob/7ba46b9d52ed0c03bf806193f38d8c0e9bd8a9b4/nltk/text.py#L531

Nltk тоже умеет считать TF. Точнее, "умеет"

In [35]:
a = nltk.text.TextCollection(texts)

In [36]:
a.tf('и', 'лавировали лавировали да не вылавировались')

0.14285714285714285

In [37]:
len('лавировали лавировали да не вылавировались')

42

In [ ]:
'лавировали лавировали да не вылавировались'.count('и')/len('лавировали лавировали да не вылавировались')

In [ ]:
3/len('корабли лавировали лавировали да не вылавировали')

In [ ]:
'корабли лавировали лавировали да не вылавировали'.count('лави')

In [ ]:
mytexts = nltk.TextCollection(texts)

In [ ]:
mytexts.tf(u'ли', 'корабли лавировали лавировали да не вылавировали')

## ---

## Будьте внимательны

In [ ]:
# TF-IDF для "мамы"
res = []
TF = 3
from math import log, sqrt
IDF = log(4/2)+1
res.append(TF*IDF)
# TF-IDF для "мыла"
TF = 1
from math import log
IDF = log(4/2)+1
res.append(TF*IDF)
# TF-IDF для "рамы"
TF = 1
from math import log
IDF = log(4/3)+1
res.append(TF*IDF)
res = np.asarray(res)
res/sqrt((res**2).sum())

In [ ]:
# TF-IDF для "мамы"
res = []
TF = 3
from math import log, sqrt
IDF = log(3/1)+1
res.append(TF*IDF)
# TF-IDF для "мыла"
TF = 1
from math import log
IDF = log(3/1)+1
res.append(TF*IDF)
# TF-IDF для "рамы"
TF = 1
from math import log
IDF = log(3/2)+1
res.append(TF*IDF)
res = np.asarray(res)
res/sqrt((res**2).sum())